In [157]:
#import for loading env variables
from dotenv import load_dotenv
import os



In [158]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")



## Now let's load a wikipedia document. 
### Langchain has standard capabilities to load many types of data to its loader. see link **[here](https://python.langchain.com/docs/modules/data_connection/document_loaders)**.

In [185]:
#now let's import the ai stuff
import openai
from langchain import OpenAI
#import vectorestore we will be using
from langchain.vectorstores import FAISS
#load the easy loader for text
from langchain.document_loaders import TextLoader
#load wikipedia loader
from langchain.document_loaders import WikipediaLoader
#now import the embedding engine
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA


In [186]:
llm = OpenAI(temperature=0.5, model_name='text-davinci-003', openai_api_key=openai_api_key)

# Load content from Wikipedia using WikipediaLoader
company_name = "Klarna"
loader = WikipediaLoader(company_name,doc_content_chars_max=100000, load_max_docs=1)
doc = loader.load()

In [ ]:
print(f"you have loaded {len(doc)} document(s)")
print(f"the document contains {len(doc[0].page_content)} characters")
print(f"here are the first 100 characters:\n{doc[0].page_content[0:100]}")

### We need to split our wikipedia doc into smaller chunks. we do this with text splitter, there are several to choose from offered by langchain. Check **[this](https://python.langchain.com/docs/modules/data_connection/document_transformers/)** out for langchain's document loader built-in integrations.

In [188]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = splitter.split_documents(doc)

### Let's check that it worked and we have multiple documents from the single wikipedia one we loaded

In [189]:
total_chars_in_docs = sum([len(x.page_content) for x in docs])
print(f"nowe have {len(docs)} documents with an average {total_chars_in_docs / len(docs):,.0f} chars per document")

nowe have 7 documents with an average 1,296 chars per document


###  Create embeddings and store in **[FAISS](https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss)** vectorestore

In [ ]:
#Get embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
#embed the documents and combine with raw text in a pseudo vectorstore. This will make a call to openai 
docsearch = FAISS.from_documents(docs, embeddings)


### Before we can ask questions about our document, we need to use a retriver. 
Read about retrivers  **[here](https://python.langchain.com/docs/modules/data_connection/retrievers/)**

In [192]:

#create the actual retriver that will help us find the best document
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=docsearch.as_retriever())




In [ ]:
#create a prompt template
from langchain.prompts import PromptTemplate

template = "Create a numbered list of {company}'s products. order the producst by {criteria}. please name the products in the list."

prompt = PromptTemplate(input_variables=["company","criteria"], template=template)
prompt= prompt.format(company=company_name, criteria="lanunch date")
#print and run the prompt
print(prompt)
qa.run(prompt)



In [ ]:
qa.run("what was the previosu question I asked you? please be percise and use the same words I used")

### Create a simple chatbot with memory

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
import os

openai_api_key= os.getenv("OPENAI_API_KEY")
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

#chat = ConversationChain(llm=llm, verbose=True, memory=ConversationBufferMemory())
chat = ConversationChain(llm=llm, verbose=True,memory=ConversationSummaryMemory(llm=llm))

chat.predict(input="Hello my name is Yaron, how are you?")
chat.predict(input="I am doing great, do you remember my name?")
chat.predict(input="what was my first sentence to you?")
chat.predict(input="what memory solution you use to know previous context of our conversation?")






###  Build a chatbot with memory and include prompt template

In [205]:
from langchain import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

#chat specific memory
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory


In [206]:
template = """
you are a very polite and helpful chatbot. You try to give the best answer and always finish with a joke incorprating an animal. 

{chat_hitsory}
Human: {human_input}
Chatbot:
"""

#create the prmpt from the template
prompt = PromptTemplate(input_variables=["chat_hitsory","human_input"], template=template)

#define the memory for the chatbot
memory = ConversationBufferMemory(memory_key="chat_hitsory")
#memory = ConversationSummaryMemory(llm=llm, memory_key="chat_hitsory")
#create the llm 
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)



In [ ]:
llm_chain = LLMChain(llm=llm,memory=memory, prompt=prompt, verbose=True)


llm_chain.predict(human_input="what is Klarna?")



In [ ]:
llm_chain.predict(human_input="which animal did was mentioned in our conversation?")

## Agents

Read the *[LangChain Agent Docs](https://python.langchain.com/en/latest/modules/agents.html)*

Agents are the decision makers that can look a data, reason about what the next action should be, and execute that action for you via tools. Langchain supports integrations to a variety of tools. Read more about the supported tools *[here](https://python.langchain.com/docs/modules/agents/tools/)*


In [131]:
import os
import json
from langchain.llms import OpenAI
from dotenv import load_dotenv

#agents related imports
from langchain.agents import load_tools
from langchain.agents import initialize_agent

#now import tools
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import TextRequestsWrapper


In [132]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
google_cse_id = os.getenv("GOOGLE_CSE_ID")

llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

### Initialise the tools the agent will use

In [133]:
search = GoogleSearchAPIWrapper()
requests = TextRequestsWrapper()

#put tools in toolkit lits
toolkit = [
    Tool(name="Search", func=search.run, description="useful when you want to seatch web for current information"),
    Tool(name="Requests", func=requests.get, description="useful when you need to make a request to a website or url"),
]

### Initialise the agent

In [134]:
#agent initialization
agent = initialize_agent(tools=toolkit, llm=llm, verbose=True, agent="zero-shot-react-description", return_intermediate_results=True)

In [ ]:
#ask the agent a question
#response = agent({"input":"what is the capital of israel?"})
response = agent("{what are famour fintechs founded in Sweden?}")
response['output']

